In [1]:
import torch
import torch.nn as nn 
import torch.nn.functional as F

class BasicBlock(nn.Module) : 
    
    expansion = 1 # 확장 비율 변수 => ResNet block 채널수를 확장하는 경우에 필요에 의해서 숫자를 늘려주시면됩니다. (기본 1)
    
    def __init__(self, in_channels, out_channels, stride=1) : 
        super(BasicBlock, self).__init__()
        
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
                
        if stride != 1 or in_channels != self.expansion * out_channels : 
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
              
            )
    
    def forward(self, x) :
        out = F.relu(self.bn1(x)) # bn1 -> relue 
        residual = out
        
        out = self.conv1(out) # relue -> conv1
        out = self.conv2(F.relu(self.bn2(out))) ### bn2 -> relue -> conv2
        
        if hasattr(self, 'shortcut')  : ### 기존 self.shortcut 을 가지고있는지 확인 체크 
            residual = self.shortcut(x)
            
        out += residual
        
        return out

In [2]:
class PreActResNet(nn.Module) : 
    def __init__(self, block, layers, num_classes=1000) :
        super(PreActResNet, self).__init__()
        
        self.in_channels = 64 
        
        self.conv1 = nn.Conv2d(3, self.in_channels,kernel_size=3, stride=1, padding=1, bias=False)
        
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        
    def _make_layer(self, block, out_channels, blocks, stride=1) : 
        layers = [] #### ->>>>> 레이어들 담을 리스트 
        layers.append(block(self.in_channels, out_channels, stride=1))
        
        self.in_channels = out_channels * block.expansion
        
        for _ in range(1, blocks) : 
            # in_channels -> 이전 설정한 출력 채널 / out_channels -> out_channels
            layers.append(block(self.in_channels, out_channels))
        
        print(layers)
        return nn.Sequential(*layers) ### nn.Sequential() -> 모델 구성하기위한 컨테이너 클래스 : 순차적으로 레이어 추가 
    
    def forward(self, x) :
        x = self.conv1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x
        

In [3]:
def pract_resnet34(num_classes = 1000) :
    return PreActResNet(BasicBlock, [3,4,6,3], num_classes)

def pract_resnet18(num_classes = 1000) :
    return PreActResNet(BasicBlock, [2,2,2,2], num_classes)

model = pract_resnet18(num_classes = 1000)
inputs = torch.randn(1,3,224,224)
output = model(inputs)
print(output.shape)

[BasicBlock(
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
), BasicBlock(
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)]
[BasicBlock(
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, t